# Dependencies
* Installing `pyarrow` is needed for parquet support

In [1]:
! pip install pyarrow


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import glob

# Reading Data
* Since our data is in parquet format, we need to loop over our data to have them in dataframes

    1. Extracting Table Names using `os.listdir`
    
    2. Creating a dictionary with keys for each table and appending table parquet files as a list for each key

    3. Reading each parquet file for each table and creating a dictionary for each dataframe with table names to concatenate them afterwards

    4. Concatenating each dataframe value and creating a dataframe with relevant name for the table and saving the dataframe as a csv file to be able to use later and share with the team

In [3]:
BASE_DIR = 'tennis_data_20231212/raw/'
folders = os.listdir(BASE_DIR)
folders

['raw_match_parquet',
 'raw_odds_parquet',
 'raw_point_by_point_parquet',
 'raw_statistics_parquet',
 'raw_tennis_power_parquet',
 'raw_votes_parquet']

## 1. Extracting Table Names using `os.listdir` and converting it to `set()` so we can have unique names

In [4]:
table_list = []
for folder in folders:
    files = os.listdir(BASE_DIR + folder)
    for file in files:
        name = file.split('.')[0]
        table_name = name.split('_')
        table_name = '_'.join(table_name[:-1])
        table_list.append(table_name)
        
table_set = set(table_list)
table_set

{'away_team',
 'away_team_score',
 'event',
 'home_team',
 'home_team_score',
 'odds',
 'pbp',
 'power',
 'round',
 'season',
 'statistics',
 'time',
 'tournament',
 'venue',
 'votes'}

## 2. Creating a dictionary with keys for each table and appending table parquet files as a list for each key

In [5]:
table_dict = {name: [] for name in list(table_set)}
    
for folder in folders:
    files = os.listdir(BASE_DIR + folder)
    for file in files:
        table_name = file.split('.')[0].split('_')
        table_name = '_'.join(table_name[:-1])
        table_dict[table_name].append(BASE_DIR + folder + '/' + file)



## 3. Reading each parquet file for each table and creating a dictionary for each dataframe with table names to concatenate them afterwards

In [6]:
dataframes_dict = {name: [] for name in list(table_set)}
for key in table_dict:
    for file in table_dict[key]:
        dataframes_dict[key].append(pd.read_parquet(file))

In [7]:
dataframes_dict['away_team']

[   match_id        name                    slug gender  user_count residence  \
 0  11673958  Tirante T.  tirante-thiago-agustin      M        1519  La Plata   
 
             birthplace  height  weight         plays turned_pro  \
 0  La Plata, Argentina    1.85      78  right-handed       None   
 
    current_prize  total_prize  player_id  current_rank name_code    country  \
 0         249335       430793     221515           123       TIR  Argentina   
 
                  full_name  
 0  Tirante, Thiago Agustin  ,
    match_id      name           slug gender  user_count residence birthplace  \
 0  11673962  Heide G.  heide-gustavo      M        1177      None  Sao Paulo   
 
    height weight         plays turned_pro  current_prize  total_prize  \
 0    1.88   None  right-handed       None          36929        87230   
 
    player_id  current_rank name_code country       full_name  
 0     302582           247       HEI  Brazil  Heide, Gustavo  ,
    match_id              name  

## 4. Concatenating each dataframe value and creating a dataframe with relevant name for the table and saving the dataframe as a csv file to be able to use later and share with the team

In [8]:
output_dir = 'tennis_csv/'
for key in dataframes_dict:
    dataframe_name = f'df_{key}'
    df = globals()[dataframe_name] = pd.concat(dataframes_dict[key], ignore_index=True)
    csv_file_name = f"{dataframe_name}.csv"
    file_path = os.path.join(output_dir, csv_file_name)
    df.to_csv(file_path, index=False)

C:\Users\arian\AppData\Local\Temp\ipykernel_27768\1441828325.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = globals()[dataframe_name] = pd.concat(dataframes_dict[key], ignore_index=True)


# Loading Data

***STEPS***
1. Loading all the dataframes from csv file

2. Cleaning Dataframes and Understanding what they present

In [9]:
INPUT_DIR = 'tennis_csv/'

## 1. Loading all the dataframes from csv file

In [10]:
for file in os.listdir(INPUT_DIR):
    df_name = file.split('.')[0] + '_csv'
    globals()[df_name] = pd.read_csv(INPUT_DIR + file)

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['df_power.csv', 'df_round.csv', 'df_season.csv', 'df_statistics.csv', 'df_time.csv', 'df_tournament.csv', 'df_venue.csv', 'df_votes.csv', 'df_away_team.csv', 'df_away_team_score.csv', 'df_event.csv', 'df_home_team.csv', 'df_home_team_score.csv', 'df_odds.csv', 'df_pbp.csv']

### 1. MatchTournamentInfo

1. `match_id`
    * **Numerical**
    * Showing ID of the match and seems like all the tables relate to each other with this ID
    
2. `tournament_id`
    * **Numerical**
    * Showing ID of the tournament for each match. 
    * Has duplicate values that differ with match ID

3. `tournament_name`
    * **Categorical**
    * This column shows tournament name and it is combination of city and country shown like this : `City, Country`
    * This column has **55 Unique Values**
4. `tournament_slug`
    * **Categorical**
    * This column shows tournament name and it is combination of city and country shown like this : `city-country`
    * This column has **55 Unique Values**

5. `tournament_unique_id`
    * **Numerical**
    * All the values of this column is missing and **This column will be droped in the Cleaning Process**
6. `tournament_category_name`
    * **Categorical**
    * This column shows the category of the tournament
    * Has **5 Unique Values**
        * Challenger
        * WTA
        * ATP
        * ITF Men
        * ITF Women

7. `tournament_category_slug`
    * **Categorical**
    * This column shows the category of the tournament in slug meaning they are lower case and have no space
    * Has **5 Unique Values**
        * challenger
        * wta
        * atp
        * itf-men
        * itf-women

8. `user_count`
    * **Numerical**
    * This column shows how many users participated in the tournament
    * Some of the information about this column:
        * Mean : 223.56
        * Min : 2
        * Max : 3627
        * std : 711.36
        * Q1 : 7
        * Q3 : 19
        

9. `ground_type`
    * **Categorical**
    * This column shows what kind of ground the match was played on
    * Has **6 Unique Values**
        * Red clay
        * Hardcourt outdoor
        * Hardcourt indoor
        * Carpet indoor
        * nan
        * Synthetic outdoor
    * **Has 17 Missing Values that will be dropped**


10. `tennis_points`
    * **Numerical**
    * This column shows how many points were scored in this tournament
    * This column has **600 Missing Values** - *Not decided what to do with it*
    * Some of the information about this column:
        * Mean : 406.097561
        * Min : 75
        * Max : 1000
        * std : 412.441871
        * Q1 : 100
        * Q3 : 1000

***All the following columns only have **One** Value and **None** is Missing***

11. `has_event_player_statistics`

12. `crowd_sourcing_enabled`

13. `has_performance_graph_feature`

14. `display_inverse_home_away_teams`

15. `priority`

16. `competition_type`


In [ ]:
df_tournament_csv.ground_type.unique()

array(['Red clay', 'Hardcourt outdoor', 'Hardcourt indoor',
       'Carpet indoor', nan, 'Synthetic outdoor'], dtype=object)

In [ ]:
df_tournament_csv.head()

,match_id,tournament_id,tournament_name,tournament_slug,tournament_unique_id,tournament_category_name,tournament_category_slug,user_count,ground_type,tennis_points,has_event_player_statistics,crowd_sourcing_enabled,has_performance_graph_feature,display_inverse_home_away_teams,priority,competition_type
0,11673958,121441,"Bogota, Colombia",bogota-colombia,NaN,Challenger,challenger,125,Red clay,125.0,False,False,False,False,0,2
1,11673962,121441,"Bogota, Colombia",bogota-colombia,NaN,Challenger,challenger,125,Red clay,125.0,False,False,False,False,0,2
2,11673963,121441,"Bogota, Colombia",bogota-colombia,NaN,Challenger,challenger,125,Red clay,125.0,False,False,False,False,0,2
3,11673993,121451,"Braga, Portugal",braga-portugal,NaN,Challenger,challenger,163,Red clay,75.0,False,False,False,False,0,2
4,11674000,121453,"Charleston, USA",charleston-usa,NaN,Challenger,challenger,71,Hardcourt outdoor,75.0,False,False,False,False,0,2


In [ ]:
df_home_team.head()

,match_id,name,slug,gender,user_count,residence,birthplace,height,weight,plays,turned_pro,current_prize,total_prize,player_id,current_rank,name_code,country,full_name
0,11673958,Barrios Vera T.,barrios-vera-tomas,M,2915,Santiago,"Chillan, Chile",1.91,83,right-handed,2014,265608,646273,132834,103,BAR,Chile,"Barrios Vera, Tomás"
1,11673962,Tirante T.,tirante-thiago-agustin,M,1519,La Plata,"La Plata, Argentina",1.85,78,right-handed,None,249335,430793,221515,123,TIR,Argentina,"Tirante, Thiago Agustin"
2,11673963,Heide G.,heide-gustavo,M,1177,None,Sao Paulo,1.88,None,right-handed,None,36929,87230,302582,247,HEI,Brazil,"Heide, Gustavo"
3,11673993,Ajduković D.,ajdukovic-duje,M,1293,"Zagreb, Croatia","Split, Croatia",1.88,75,right-handed,None,63678,198866,207081,143,AJD,Croatia,"Ajduković, Duje"
4,11674000,Escobedo E.,escobedo-ernesto,M,680,"West Covina, CA, USA","Los Angeles, CA, USA",1.85,82,right-handed,2014,84375,1280430,80491,404,ESC,Mexico,"Escobedo, Ernesto"


In [ ]:
df_event.head()

,match_id,first_to_serve,home_team_seed,away_team_seed,custom_id,winner_code,default_period_count,start_datetime,match_slug,final_result_only
0,11673958,None,1,4,JgdbspFNb,2,3,1696094700,tirante-barrios-vera,False
1,11673962,None,4,None,pFNbsHbwc,1,3,1696176000,heide-tirante,False
2,11673963,None,None,None,fTxsHbwc,1,3,1696087800,heide-jorda-sanchis,False
3,11673993,None,None,Q,axvsGQHb,2,3,1696154400,ajdukovic-roca-batalla,False
4,11674000,None,Q,WC,QjHshlKb,2,3,1696086000,crawford-escobedo,False


In [ ]:
df_tournament[df_tournament['tournament_id'] == 121453]

,match_id,tournament_id,tournament_name,tournament_slug,tournament_unique_id,tournament_category_name,tournament_category_slug,user_count,ground_type,tennis_points,has_event_player_statistics,crowd_sourcing_enabled,has_performance_graph_feature,display_inverse_home_away_teams,priority,competition_type
4,11674000,121453,"Charleston, USA",charleston-usa,None,Challenger,challenger,71,Hardcourt outdoor,75,False,False,False,False,0,2
5,11674008,121453,"Charleston, USA",charleston-usa,None,Challenger,challenger,71,Hardcourt outdoor,75,False,False,False,False,0,2
6,11674011,121453,"Charleston, USA",charleston-usa,None,Challenger,challenger,71,Hardcourt outdoor,75,False,False,False,False,0,2


In [ ]:
df_tournament_csv.columns

Index(['match_id', 'tournament_id', 'tournament_name', 'tournament_slug',
       'tournament_unique_id', 'tournament_category_name',
       'tournament_category_slug', 'user_count', 'ground_type',
       'tennis_points', 'has_event_player_statistics',
       'crowd_sourcing_enabled', 'has_performance_graph_feature',
       'display_inverse_home_away_teams', 'priority', 'competition_type'],
      dtype='object')

In [ ]:
df_tournament_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   match_id                         764 non-null    int64  
 1   tournament_id                    764 non-null    int64  
 2   tournament_name                  764 non-null    object 
 3   tournament_slug                  764 non-null    object 
 4   tournament_unique_id             0 non-null      float64
 5   tournament_category_name         764 non-null    object 
 6   tournament_category_slug         764 non-null    object 
 7   user_count                       764 non-null    int64  
 8   ground_type                      747 non-null    object 
 9   tennis_points                    164 non-null    float64
 10  has_event_player_statistics      764 non-null    bool   
 11  crowd_sourcing_enabled           764 non-null    bool   
 12  has_performance_graph_

In [ ]:
df_tournament_csv.describe()

,match_id,tournament_id,tournament_unique_id,user_count,tennis_points,priority,competition_type
count,7.640000e+02,764.000000,0.0,764.000000,164.000000,764.0,764.0
mean,1.169902e+07,121710.625654,NaN,223.560209,406.097561,0.0,2.0
std,5.029278e+03,538.709546,NaN,711.361346,412.441871,0.0,0.0
min,1.167396e+07,119387.000000,NaN,2.000000,75.000000,0.0,2.0
25%,1.169901e+07,121843.000000,NaN,7.000000,100.000000,0.0,2.0
50%,1.169935e+07,121856.000000,NaN,11.000000,100.000000,0.0,2.0
75%,1.170151e+07,121870.000000,NaN,19.000000,1000.000000,0.0,2.0
max,1.170352e+07,121898.000000,NaN,3627.000000,1000.000000,0.0,2.0


In [ ]:
df_tournament_csv.isna().sum()

match_id                             0
tournament_id                        0
tournament_name                      0
tournament_slug                      0
tournament_unique_id               764
tournament_category_name             0
tournament_category_slug             0
user_count                           0
ground_type                         17
tennis_points                      600
has_event_player_statistics          0
crowd_sourcing_enabled               0
has_performance_graph_feature        0
display_inverse_home_away_teams      0
priority                             0
competition_type                     0
dtype: int64

In [ ]:
df_tournament_csv.nunique()

match_id                           764
tournament_id                       56
tournament_name                     55
tournament_slug                     55
tournament_unique_id                 0
tournament_category_name             5
tournament_category_slug             5
user_count                          31
ground_type                          5
tennis_points                        6
has_event_player_statistics          1
crowd_sourcing_enabled               1
has_performance_graph_feature        1
display_inverse_home_away_teams      1
priority                             1
competition_type                     1
dtype: int64

### 1. MatchVenueInfo

1. `match_id`
    * **Numerical**
    * Showing ID of the match and seems like all the tables relate to each other with this ID
    
2. `city`
    * **Categorical**
    * Showing City for each match. 
    * This column has **38 Unique Values**

3. `stadium`
    * **Categorical**
    * This column shows stadium name
    * This column has **43 Unique Values**

4. `venue_id`
    * **Numerical**
    * Showing ID of the venue

5. `country`
    * **Categorical**
    * This column has **One** Missing Value


In [ ]:
df_venue.head()

,match_id,city,stadium,venue_id,country
0,11673958,Bogota,Cancha Central,18338,Colombia
1,11673962,Bogota,Cancha Central,18338,Colombia
2,11673963,Bogota,Cancha Central,18338,Colombia
3,11673993,Braga,Court Central,21510,Portugal
4,11674000,Charleston,Center Court,32522,USA


In [ ]:
df_venue.columns

Index(['match_id', 'city', 'stadium', 'venue_id', 'country'], dtype='object')

In [ ]:
df_venue.stadium.nunique()

43

In [ ]:
df_venue.city.nunique()

38

In [ ]:
df_venue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 763 entries, 0 to 762
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   match_id  763 non-null    int64 
 1   city      763 non-null    object
 2   stadium   763 non-null    object
 3   venue_id  763 non-null    int64 
 4   country   762 non-null    object
dtypes: int64(2), object(3)
memory usage: 29.9+ KB


In [ ]:
df_venue.describe()

,match_id,venue_id
count,7.630000e+02,763.000000
mean,1.169904e+07,27783.026212
std,5.017576e+03,8823.835485
min,1.167396e+07,2102.000000
25%,1.169901e+07,19072.500000
50%,1.169935e+07,28799.000000
75%,1.170152e+07,34383.000000
max,1.170352e+07,41921.000000


In [ ]:
df_venue.isna().sum()

match_id    0
city        0
stadium     0
venue_id    0
country     1
dtype: int64

### 1. MatchVenueInfo

1. `match_id`
    * **Numerical**
    * Showing ID of the match and seems like all the tables relate to each other with this ID
    
2. `home_vote`
    * **Numerical**
    * This column shows count of home_votes
    * This column has **38 Unique Values**
    * Some of the information about this column:
        * Mean : 136.298429
        * Min : 0
        * Max : 8133
        * std : 591.036442
        * Q1 : 2
        * Q3 : 32

3. `away_vote`
    * **Numerical**
    * This column shows count of away_votes
    * This column has **43 Unique Values**
    * Some of the information about this column:
        * Mean : 130.712042
        * Min : 0
        * Max : 6851
        * std : 521.944636
        * Q1 : 2
        * Q3 : 24



In [ ]:
df_votes.head()

,match_id,home_vote,away_vote
0,11673958,554,233
1,11673962,909,213
2,11673963,399,213
3,11673993,744,208
4,11674000,461,320


In [ ]:
df_votes.columns

Index(['match_id', 'home_vote', 'away_vote'], dtype='object')

In [ ]:
df_votes.describe()

,match_id,home_vote,away_vote
count,7.640000e+02,764.000000,764.000000
mean,1.169902e+07,136.298429,130.712042
std,5.029278e+03,591.036442,521.944636
min,1.167396e+07,0.000000,0.000000
25%,1.169901e+07,2.000000,2.000000
50%,1.169935e+07,7.000000,5.000000
75%,1.170151e+07,32.250000,24.000000
max,1.170352e+07,8133.000000,6851.000000


In [ ]:
df_votes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764 entries, 0 to 763
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype
---  ------     --------------  -----
 0   match_id   764 non-null    int64
 1   home_vote  764 non-null    int64
 2   away_vote  764 non-null    int64
dtypes: int64(3)
memory usage: 18.0 KB


### 1. MatchTournamentInfo

1. `match_id`
    * **Numerical**
    * Showing ID of the match and seems like all the tables relate to each other with this ID
    
2. `name`
    * **Categorical**
    * Showing home team player name. 
    * Has 89 duplicate values that differ with match ID

3. `slug`
    * **Categorical**
    * Showing home team player name. 
    * Has 89 duplicate values that differ with match ID
    
4. `gender`
    * **Categorical**
    * This column shows gender of the players
    * This column has two unique values of **F** and **M**

5. `user_count`
    * **Numerical**
    * NO IDEA

6. `residence`
    * **Categorical**
    * Shows the city the player resides in
    * This column has 85 unique values

7. `birthplace`
    * **Categorical**
    * Shows the city the player was born in
    * This column has 188 unique values

8. `user_count`
    * **Numerical**
    * This column shows how many users participated in the tournament
    * Some of the information about this column:
        * Mean : 223.56
        * Min : 2
        * Max : 3627
        * std : 711.36
        * Q1 : 7
        * Q3 : 19
        

9. `height`
    * **Numerical**
    * This column shows height of the player
    * Has **6 Unique Values**
        * Red clay
        * Hardcourt outdoor
        * Hardcourt indoor
        * Carpet indoor
        * nan
        * Synthetic outdoor
    * **Has 17 Missing Values that will be dropped**


10. `tennis_points`
    * **Numerical**
    * This column shows how many points were scored in this tournament
    * This column has **600 Missing Values** - *Not decided what to do with it*
    * Some of the information about this column:
        * Mean : 406.097561
        * Min : 75
        * Max : 1000
        * std : 412.441871
        * Q1 : 100
        * Q3 : 1000

***All the following columns only have **One** Value and **None** is Missing***

11. `has_event_player_statistics`

12. `crowd_sourcing_enabled`

13. `has_performance_graph_feature`

14. `display_inverse_home_away_teams`

15. `priority`

16. `competition_type`


In [3]:
import pandas as pd

In [4]:
df_home_team = pd.read_csv('tennis_csv/df_home_team.csv')

In [11]:
df_home_team.head()

,match_id,name,slug,gender,user_count,residence,birthplace,height,weight,plays,turned_pro,current_prize,total_prize,player_id,current_rank,name_code,country,full_name
0,11673958,Barrios Vera T.,barrios-vera-tomas,M,2915,Santiago,"Chillan, Chile",1.91,83,right-handed,2014,265608,646273,132834,103,BAR,Chile,"Barrios Vera, Tomás"
1,11673962,Tirante T.,tirante-thiago-agustin,M,1519,La Plata,"La Plata, Argentina",1.85,78,right-handed,None,249335,430793,221515,123,TIR,Argentina,"Tirante, Thiago Agustin"
2,11673963,Heide G.,heide-gustavo,M,1177,None,Sao Paulo,1.88,None,right-handed,None,36929,87230,302582,247,HEI,Brazil,"Heide, Gustavo"
3,11673993,Ajduković D.,ajdukovic-duje,M,1293,"Zagreb, Croatia","Split, Croatia",1.88,75,right-handed,None,63678,198866,207081,143,AJD,Croatia,"Ajduković, Duje"
4,11674000,Escobedo E.,escobedo-ernesto,M,680,"West Covina, CA, USA","Los Angeles, CA, USA",1.85,82,right-handed,2014,84375,1280430,80491,404,ESC,Mexico,"Escobedo, Ernesto"


In [12]:
df_home_team.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436 entries, 0 to 435
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   match_id       436 non-null    int64  
 1   name           436 non-null    object 
 2   slug           436 non-null    object 
 3   gender         436 non-null    object 
 4   user_count     436 non-null    int64  
 5   residence      112 non-null    object 
 6   birthplace     245 non-null    object 
 7   height         237 non-null    float64
 8   weight         112 non-null    object 
 9   plays          181 non-null    object 
 10  turned_pro     65 non-null     object 
 11  current_prize  427 non-null    object 
 12  total_prize    432 non-null    object 
 13  player_id      436 non-null    int64  
 14  current_rank   430 non-null    object 
 15  name_code      436 non-null    object 
 16  country        435 non-null    object 
 17  full_name      436 non-null    object 
dtypes: float64

In [8]:
df_home_team.birthplace.nunique()

188

In [9]:
x = {'Acer100': {'Chromebook Tab 10': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2018, March'},
    {'Status': 'Available. Released 2018, July'}],
   'Body': [{'Dimensions': '238.3 x 172.2 x 9.9 mm (9.38 x 6.78 x 0.39 in)'},
    {'Weight': '544.3 g (1.20 lb)'},
    {'SIM': 'No'},
    {'SIM': 'Stylus'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '9.7 inches, 291.4 cm2(~71.0% screen-to-body ratio)'},
    {'Resolution': '1536 x 2048 pixels, 4:3 ratio (~264 ppi density)'}],
   'Platform': [{'OS': 'Chrome OS'},
    {'Chipset': 'Rockchip RK3399'},
    {'CPU': 'Hexa-core (4x Cortex-A53 & 2x Cortex-A72)'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '32GB 4GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': '720p'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n/ac, dual-band, Wi-Fi Direct'},
    {'Bluetooth': '4.1, A2DP, LE'},
    {'Positioning': ''},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'USB Type-C 3.1'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Po 4500 mAh, non-removable (34 Wh)'},
    {'Talk time': 'Up to 9 h (multimedia)'}],
   'Misc': [{'Colors': 'Black, Blue'}, {'Price': 'About 330 EUR'}]},
  'Iconia Talk S': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
    {'3G bands': 'HSDPA 850 / 1900 / 2100'},
    {'4G bands': '1, 3, 7, 8, 20'},
    {'4G bands': '3, 7, 8, 28'},
    {'Speed': 'HSPA 42.2/11.5 Mbps, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2016, August. Released 2016, October'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '191.7 x 101 x 9.4 mm (7.55 x 3.98 x 0.37 in)'},
    {'Weight': '260 g (9.17 oz)'},
    {'SIM': 'Dual SIM (Micro-SIM/Nano-SIM)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '7.0 inches, 135.1 cm2(~69.8% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~210 ppi density)'}],
   'Platform': [{'OS': 'Android 6.0 (Marshmallow)'},
    {'Chipset': 'Mediatek MT8735'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP2'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '16GB 2GB RAM, 32GB 2GB RAM'}],
   'Main Camera': [{'Single': '13 MP, AF'},
    {'Features': 'HDR, panorama'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': '720p'}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 a/b/g/n, Wi-Fi Direct'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS, GLONASS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 3400 mAh, non-removable (12.92 Wh)'},
    {'Talk time': 'Up to 9 h (multimedia)'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 170 EUR'}]},
  'Liquid Z6 Plus': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2016, August. Released 2016, December'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '153.8 x 75.6 x 8.5 mm (6.06 x 2.98 x 0.33 in)'},
    {'Weight': '169 g (5.96 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Hybrid Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.5 inches, 83.4 cm2(~71.7% screen-to-body ratio)'},
    {'Resolution': '1080 x 1920 pixels, 16:9 ratio (~401 ppi density)'}],
   'Platform': [{'OS': 'Android 6.0 (Marshmallow)'},
    {'Chipset': 'Mediatek MT6753 (28 nm)'},
    {'CPU': 'Octa-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP3'}],
   'Memory': [{'Card slot': 'microSDXC (uses shared SIM slot)'},
    {'Internal': '32GB 3GB RAM'}],
   'Main Camera': [{'Single': '13 MP, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '5 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, Wi-Fi Direct'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Fingerprint (front-mounted), accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Po 4080 mAh, non-removable'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 250 EUR'}]},
  'Liquid Z6': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2016, August. Released 2016, December'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '145.5 x 72.5 x 8.5 mm (5.73 x 2.85 x 0.33 in)'},
    {'Weight': '126 g (4.44 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~65.3% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'}],
   'Platform': [{'OS': 'Android 6.0 (Marshmallow)'},
    {'Chipset': 'Mediatek MT6737 (28 nm)'},
    {'CPU': 'Quad-core 1.25 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP2'}],
   'Memory': [{'Card slot': 'microSDXC'},
    {'Internal': '8GB 1GB RAM'},
    {'Internal': 'eMMC 5.0'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2000 mAh, removable'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 120 EUR'}]},
  'Iconia Tab 10 A3-A40': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2016, April. Released 2016, June'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '259 x 167 x 8.9 mm (10.20 x 6.57 x 0.35 in)'},
    {'Weight': '-'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '10.1 inches, 295.8 cm2(~68.4% screen-to-body ratio)'},
    {'Resolution': '1920 x 1200 pixels, 16:10 ratio (~224 ppi density)'}],
   'Platform': [{'OS': 'Android 6.0 (Marshmallow)'},
    {'Chipset': 'Mediatek MT8163A'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720 MP2'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '16GB 2GB RAM, 32GB 2GB RAM, 64GB 2GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers (4 speakers)'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 a/b/g/n/ac, dual-band'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': ''},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}, {'Sensors': 'HDMI'}],
   'Battery': [{'Type': 'Li-Ion 6100 mAh, non-removable'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 230 EUR'}]},
  'Liquid X2': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900'},
    {'3G bands': 'HSDPA 900 / 1900 / 2100 - Europe, Taiwan'},
    {'3G bands': 'HSDPA 850 / 1700(AWS) / 1900 - PA'},
    {'4G bands': 'LTE 800 / 1800 / 2100 / 2600 - Europe'},
    {'4G bands': 'LTE 700 / 900 / 1800 - Taiwan'},
    {'Speed': 'HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2015, April. Released 2016, February'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '153.3 x 78.8 x 8.5 mm (6.04 x 3.10 x 0.33 in)'},
    {'Weight': '166 g (5.86 oz)'},
    {'SIM': 'Triple SIM (Micro-SIM)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.5 inches, 83.4 cm2(~69.0% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~267 ppi density)'}],
   'Platform': [{'OS': 'Android 5.1 (Lollipop)'},
    {'Chipset': 'Mediatek MT6753 (28 nm)'},
    {'CPU': 'Octa-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP4'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '32GB 3GB RAM'}],
   'Main Camera': [{'Single': '13 MP, f/1.8, AF'},
    {'Features': 'LED flash'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '13 MP, f/1.8, AF'},
    {'Features': 'LED flash'},
    {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'},
    {'3.5mm jack': 'Yes'},
    {'3.5mm jack': '24-bit/192kHz audio'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP, LE'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Po 4020 mAh, removable'},
    {'Stand-by': 'Up to 820 h (3G)'},
    {'Talk time': 'Up to 23 h (3G)'}],
   'Misc': [{'Colors': 'Black, Gold'}, {'Price': 'About 230 EUR'}],
   'Tests': [{'Performance': 'Basemark OS II 2.0: 825Basemark X: 4300'},
    {'Display': 'Contrast ratio: 950 (nominal), 2.084 (sunlight)'},
    {'Camera': 'Photo/Video'},
    {'Loudspeaker': 'Voice 61dB / Noise 62dB / Ring 67dB'},
    {'Audio quality': 'Noise -86.1dB / Crosstalk -88.1dB'},
    {'Battery (old)': 'Endurance rating 85h'}]},
  'Liquid Jade 2': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA 42.2/5.76 Mbps, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2016, February'}, {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '156.6 x 75.9 x 8.4 mm (6.17 x 2.99 x 0.33 in)'},
    {'Weight': '150 g (5.29 oz)'},
    {'SIM': 'Dual SIM (Nano-SIM, dual stand-by)'}],
   'Display': [{'Type': 'Super AMOLED'},
    {'Size': '5.5 inches, 83.4 cm2(~70.2% screen-to-body ratio)'},
    {'Resolution': '1080 x 1920 pixels, 16:9 ratio (~401 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 4'}],
   'Platform': [{'OS': 'Android 6.0 (Marshmallow)'},
    {'Chipset': 'Qualcomm MSM8992 Snapdragon 808 (20 nm)'},
    {'CPU': 'Hexa-core (4x1.4 GHz Cortex-A53 & 2x1.8 GHz Cortex-A57)'},
    {'GPU': 'Adreno 418'}],
   'Memory': [{'Card slot': 'microSDXC'},
    {'Internal': '32GB 3GB RAM'},
    {'Internal': 'eMMC 5.0'}],
   'Main Camera': [{'Single': '21 MP, AF'},
    {'Features': 'Dual-LED flash, HDR, panorama'},
    {'Video': '4K@30fps, 1080p@30fps'}],
   'Selfie camera': [{'Single': '8 MP'}, {'Video': '1080p@30fps'}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-Fi Direct'},
    {'Bluetooth': '4.0, A2DP, LE'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, gyro, proximity, compass'}],
   'Battery': [{'Type': 'Li-Po 3000 mAh, non-removable'}],
   'Misc': [{'Colors': 'Black'}]},
  'Liquid Zest Plus': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2016, April. Released 2016, July'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '154 x 77 x 10 mm (6.06 x 3.03 x 0.39 in)'},
    {'Weight': '-'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.5 inches, 83.4 cm2(~70.3% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~267 ppi density)'}],
   'Platform': [{'OS': 'Android 6.0 (Marshmallow)'},
    {'Chipset': 'Mediatek MT6735 (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP2'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '16GB 2GB RAM'}],
   'Main Camera': [{'Single': '13 MP, PDAF, Laser AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '5 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Ion 5000 mAh, non-removable'},
    {'Charging': '10W wired'}],
   'Misc': [{'Colors': 'Blue, White'}, {'Price': 'About 200 EUR'}]},
  'Liquid Zest': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2016, February. Released 2016, April'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '145.7 x 71.2 x 8.4 mm (5.74 x 2.80 x 0.33 in)'},
    {'Weight': '125 g (4.41 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~66.4% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'}],
   'Platform': [{'OS': 'Android 6.0 (Marshmallow)'},
    {'Chipset': 'Mediatek MT6580 - Z525Mediatek MT6735 - Z525'},
    {'CPU': 'Quad-core 1.3 GHz - Z525Quad-core 1.0 GHz - Z528'},
    {'GPU': 'Mali-400MP2 - Z525Mali-T720MP2 - Z528'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM'}],
   'Main Camera': [{'Single': '8 MP, f/2.0, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '5 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Ion 2000 mAh, removable'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 110 EUR'}]},
  'Predator 8': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2015, September. Released 2015, November'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '217.9 x 127 x 8.6 mm (8.58 x 5.0 x 0.34 in)'},
    {'Weight': '353.8 g (12.49 oz)'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '8.0 inches, 185.6 cm2(~67.1% screen-to-body ratio)'},
    {'Resolution': '1920 x 1200 pixels, 16:10 ratio (~283 ppi density)'}],
   'Platform': [{'OS': 'Android 5.0 (Lollipop)'},
    {'Chipset': 'Intel Atom x7-Z8700'},
    {'CPU': 'Quad-core 1.6 GHz'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '32GB 2GB RAM, 64GB 2GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers (4 speakers)'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 a/b/g/n'},
    {'Bluetooth': '4.0, A2DP, LE'},
    {'Positioning': ''},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Yes'}],
   'Battery': [{'Type': 'Li-Po 4420 mAh, non-removable'},
    {'Talk time': 'Up to 8 h 30 min'}],
   'Misc': [{'Colors': 'Black/Silver'},
    {'Models': 'GT-810-15NC'},
    {'Price': 'About 350 EUR'}]},
  'Liquid Jade Primo': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100 - Europe/ Taiwan'},
    {'3G bands': 'HSDPA 850 / 1700(AWS) / 1900 / 2100 - USA'},
    {'4G bands': 'LTE 800 / 1800 / 2100 / 2600 - Europe'},
    {'4G bands': 'LTE 900 / 1800 / 2600 - Taiwan'},
    {'4G bands': 'LTE 700 / 1700 / 1900 / 2600 - USA'},
    {'Speed': 'HSPA 42.2/5.76 Mbps, LTE-A (2CA) Cat6 300/50 Mbps'}],
   'Launch': [{'Announced': '2015, September. Released 2016, April'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '156.5 x 75.9 x 8.4 mm (6.16 x 2.99 x 0.33 in)'},
    {'Weight': '150 g (5.29 oz)'},
    {'SIM': 'Hybrid Dual SIM (Nano-SIM, dual stand-by)'}],
   'Display': [{'Type': 'AMOLED'},
    {'Size': '5.5 inches, 83.4 cm2(~70.2% screen-to-body ratio)'},
    {'Resolution': '1080 x 1920 pixels, 16:9 ratio (~401 ppi density)'},
    {'Protection': 'Corning Gorilla Glass (unspecified version)'}],
   'Platform': [{'OS': 'Microsoft Windows 10'},
    {'Chipset': 'Qualcomm MSM8992 Snapdragon 808 (20 nm)'},
    {'CPU': 'Hexa-core (4x1.4 GHz Cortex-A53 & 2x1.8 GHz Cortex-A57)'},
    {'GPU': 'Adreno 418'}],
   'Memory': [{'Card slot': 'microSDXC (uses shared SIM slot)'},
    {'Internal': '32GB 3GB RAM'},
    {'Internal': 'eMMC 5.0'}],
   'Main Camera': [{'Single': '21 MP, f/2.2, AF'},
    {'Features': 'Dual-LED flash, HDR, panorama'},
    {'Video': '4K@30fps, 1080p@30fps'}],
   'Selfie camera': [{'Single': '8 MP, f/2.2'}, {'Video': '1080p@30fps'}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 a/b/g/n/ac, dual-band, Wi-Fi Direct, hotspot'},
    {'Bluetooth': '4.0, A2DP, LE, EDR'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'USB Type-C 3.1'}],
   'Features': [{'Sensors': 'Accelerometer, gyro, proximity, compass'},
    {'Sensors': 'Microsoft Continuum support'}],
   'Battery': [{'Type': 'Li-Po 2870 mAh, non-removable'},
    {'Stand-by': 'Up to 330 h (3G)'},
    {'Talk time': 'Up to 20 h (3G)'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 220 EUR'}],
   'Tests': [{'Camera': 'Photo/Video'},
    {'Battery (old)': 'Endurance rating 56h'}]},
  'Liquid Z330': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2015, September. Released 2015, October'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '136 x 66.5 x 9.6 mm (5.35 x 2.62 x 0.38 in)'},
    {'Weight': '142 g (5.01 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.5 inches, 55.8 cm2(~61.7% screen-to-body ratio)'},
    {'Resolution': '480 x 854 pixels, 16:9 ratio (~218 ppi density)'}],
   'Platform': [{'OS': 'Android 5.1 (Lollipop)'},
    {'Chipset': 'Qualcomm MSM8909 Snapdragon 210 (28 nm)'},
    {'CPU': 'Quad-core 1.1 GHz Cortex-A7'},
    {'GPU': 'Adreno 304'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM'},
    {'Internal': 'eMMC 4.5'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '5 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2000 mAh, removable'},
    {'Stand-by': 'Up to 430 h'},
    {'Talk time': 'Up to 11 h'}],
   'Misc': [{'Colors': 'Mystic Black, Pure White'},
    {'Price': 'About 110 EUR'}]},
  'Liquid Z320': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA 42.2/5.76 Mbps'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2015, September. Released 2015, November'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '136 x 66.5 x 9.6 mm (5.35 x 2.62 x 0.38 in)'},
    {'Weight': '142 g (5.01 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.5 inches, 55.8 cm2(~61.7% screen-to-body ratio)'},
    {'Resolution': '480 x 854 pixels, 16:9 ratio (~218 ppi density)'}],
   'Platform': [{'OS': 'Android 5.1 (Lollipop)'},
    {'Chipset': 'Qualcomm MSM8909 Snapdragon 210 (28 nm)'},
    {'CPU': 'Quad-core 1.0 GHz Cortex-A7'},
    {'GPU': 'Adreno 304'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM'},
    {'Internal': 'eMMC 4.5'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Po 2000 mAh, removable'}],
   'Misc': [{'Colors': 'Pure White'}, {'Price': 'About 90 EUR'}]},
  'Liquid Z630S': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 900 / 1900 / 2100'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'}],
   'Launch': [{'Announced': '2015, September. Released 2015, September'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '156.3 x 77.5 x 8.9 mm (6.15 x 3.05 x 0.35 in)'},
    {'Weight': '165 g (5.82 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.5 inches, 83.4 cm2(~68.8% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~267 ppi density)'}],
   'Platform': [{'OS': 'Android 5.1 (Lollipop)'},
    {'Chipset': 'Mediatek MT6753 (28 nm)'},
    {'CPU': 'Octa-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP3'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '32GB 3GB RAM'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '8 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0, OTG'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Po 4000 mAh, removable (15.2 Wh)'}],
   'Misc': [{'Colors': 'Black, Gold'}]},
  'Liquid Z630': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2015, September. Released 2015, September'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '156.3 x 77.5 x 8.9 mm (6.15 x 3.05 x 0.35 in)'},
    {'Weight': '165 g (5.82 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.5 inches, 83.4 cm2(~68.8% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~267 ppi density)'}],
   'Platform': [{'OS': 'Android 5.1 (Lollipop)'},
    {'Chipset': 'Mediatek MT6735 (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP2'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM, 16GB 2GB RAM'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '8 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0, OTG'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Ion 4000 mAh, removable'},
    {'Stand-by': 'Up to 1030 h'},
    {'Talk time': 'Up to 22 h'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 200 EUR'}]},
  'Liquid Z530S': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2015, September. Released 2015, November'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '144 x 70.3 x 8.9 mm (5.67 x 2.77 x 0.35 in)'},
    {'Weight': '145 g (5.11 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~68.1% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'}],
   'Platform': [{'OS': 'Android 5.1 (Lollipop)'},
    {'Chipset': 'Mediatek MT6753 (28 nm)'},
    {'CPU': 'Octa-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP3'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '32GB 3GB RAM'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '8 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2420 mAh, removable'}],
   'Misc': [{'Colors': 'Black, White'}]},
  'Liquid Z530': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2015, September. Released 2015, October'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '144 x 70.3 x 8.9 mm (5.67 x 2.77 x 0.35 in)'},
    {'Weight': '145 g (5.11 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~68.1% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'}],
   'Platform': [{'OS': 'Android 5.1 (Lollipop)'},
    {'Chipset': 'Mediatek MT6735 (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T720MP2'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM, 16GB 2GB RAM'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '8 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2420 mAh, removable'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 150 EUR'}]},
  'Liquid M330': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'4G bands': 'LTE (unspecified)'},
    {'Speed': 'HSPA, LTE'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2015, September. Released 2015, November'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '136 x 66.5 x 9.6 mm (5.35 x 2.62 x 0.38 in)'},
    {'Weight': '142 g (5.01 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.5 inches, 55.8 cm2(~61.7% screen-to-body ratio)'},
    {'Resolution': '480 x 854 pixels, 16:9 ratio (~218 ppi density)'}],
   'Platform': [{'OS': 'Microsoft Windows 10'},
    {'Chipset': 'Qualcomm MSM8909 Snapdragon 210 (28 nm)'},
    {'CPU': 'Quad-core 1.0 GHz Cortex-A7'},
    {'GPU': 'Adreno 304'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM'},
    {'Internal': 'eMMC 4.5'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '5 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2000 mAh, removable'}],
   'Misc': [{'Colors': 'Mystic Black, Pure White'}]},
  'Liquid M320': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA 42.2/5.76 Mbps'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2015, September. Released 2015, November'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '136 x 66.5 x 9.6 mm (5.35 x 2.62 x 0.38 in)'},
    {'Weight': '142 g (5.01 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.5 inches, 55.8 cm2(~61.7% screen-to-body ratio)'},
    {'Resolution': '480 x 854 pixels, 16:9 ratio (~218 ppi density)'}],
   'Platform': [{'OS': 'Microsoft Windows 10'},
    {'Chipset': 'Qualcomm MSM8909 Snapdragon 210 (28 nm)'},
    {'CPU': 'Quad-core 1.1 GHz Cortex-A7'},
    {'GPU': 'Adreno 304'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM'},
    {'Internal': 'eMMC 4.5'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2000 mAh, removable'}],
   'Misc': [{'Colors': 'Mystic Black, Pure White'}]},
  'Iconia Tab 10 A3-A30': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2015, April. Released 2015, June'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '260 x 176 x 9.7 mm (10.24 x 6.93 x 0.38 in)'},
    {'Weight': '540 g (1.19 lb)'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'LED-backlit IPS LCD'},
    {'Size': '10.1 inches, 295.8 cm2(~64.6% screen-to-body ratio)'},
    {'Resolution': '1920 x 1200 pixels, 16:10 ratio (~224 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 4, oleophobic coating'}],
   'Platform': [{'OS': 'Android 5.0 (Lollipop)'},
    {'Chipset': 'Intel Atom Z3735F'},
    {'CPU': 'Quad-core 1.33 GHz'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '16GB 2GB RAM, 32GB 2GB RAM, 64GB 2GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 a/b/g/n/ac, dual-band, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'A-GPS only'},
    {'NFC': 'Yes'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Ion 5910 mAh, non-removable (22 Wh)'},
    {'Talk time': 'Up to 7 h 30 min (multimedia)'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 250 EUR'}]},
  'Iconia One 8 B1-820': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2015, April. Released 2015, May'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '213.9 x 127.7 x 9.5 mm (8.42 x 5.03 x 0.37 in)'},
    {'Weight': '355 g (12.52 oz)'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '8.0 inches, 185.6 cm2(~67.9% screen-to-body ratio)'},
    {'Resolution': '800 x 1280 pixels, 16:10 ratio (~189 ppi density)'}],
   'Platform': [{'OS': 'Android 5.0 (Lollipop)'},
    {'Chipset': 'Intel Atom Z3735G'},
    {'CPU': 'Quad-core 1.33 GHz'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '16GB 1GB RAM, 32GB 1GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': 'VGA'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS, GLONASS'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Ion 4550 mAh, non-removable (17.2 Wh)'},
    {'Talk time': 'Up to 8 h 30 min (multimedia)'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 180 EUR'}]},
  'Iconia Tab A3-A20': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2014, October. Released 2014, October'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '256.5 x 170.2 x 10.2 mm (10.10 x 6.70 x 0.40 in)'},
    {'Weight': '508 g (1.12 lb)'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'LED-backlit IPS LCD'},
    {'Size': '10.1 inches, 295.8 cm2(~67.8% screen-to-body ratio)'},
    {'Resolution': '1280 x 800 pixels, 16:10 ratio (~149 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 4, oleophobic coating'}],
   'Platform': [{'OS': 'Android 4.4 (KitKat)'},
    {'Chipset': 'Mediatek MT8127 (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A7'},
    {'GPU': 'Mali-450MP4'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '16GB 1GB RAM'}],
   'Main Camera': [{'Single': '5 MP, AF'}, {'Video': '720p'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'A-GPS only'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Po 5910 mAh, non-removable (A3-A20-K1AY)'},
    {'Talk time': 'Up to 9 h (multimedia)'},
    {'Stand-by': ''},
    {'Talk time': 'Up to 9 h (multimedia)'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 190 EUR'}]},
  'Iconia Tab A3-A20FHD': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2014, October. Released 2014, October'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '256.5 x 170.2 x 10.2 mm (10.10 x 6.70 x 0.40 in)'},
    {'Weight': '508 g (1.12 lb)'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'LED-backlit IPS LCD'},
    {'Size': '10.1 inches, 295.8 cm2(~67.8% screen-to-body ratio)'},
    {'Resolution': '1920 x 1200 pixels, 16:10 ratio (~224 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 4, oleophobic coating'}],
   'Platform': [{'OS': 'Android 4.4 (KitKat)'},
    {'Chipset': 'Mediatek MT8127 (28 nm)'},
    {'CPU': 'Quad-core 1.5 GHz Cortex-A7'},
    {'GPU': 'Mali-450MP4'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '32GB 2GB RAM'}],
   'Main Camera': [{'Single': '5 MP, AF'}, {'Video': '720p'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'A-GPS only'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Po 5910 mAh, non-removable (A3-A20FHD-K8KX)'},
    {'Talk time': 'Up to 8 h (multimedia)'},
    {'Stand-by': ''},
    {'Talk time': 'Up to 8 h (multimedia)'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 230 EUR'}]},
  'Liquid Jade Z': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'4G bands': '1, 3, 7, 20'},
    {'Speed': 'HSPA 21.1/5.76 Mbps, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2015, March. Released 2015, April'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '143.5 x 69.8 x 8 mm (5.65 x 2.75 x 0.31 in)'},
    {'Weight': '110 g (3.88 oz)'},
    {'SIM': 'Hybrid Dual SIM (Nano-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~68.8% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 3'}],
   'Platform': [{'OS': 'Android 5.0 (Lollipop)'},
    {'Chipset': 'Mediatek MT6732 (28 nm)'},
    {'CPU': 'Quad-core 1.5 GHz Cortex-A53'},
    {'GPU': 'Mali-T760MP2'}],
   'Memory': [{'Card slot': 'microSDHC (uses shared SIM slot)'},
    {'Internal': '8GB 1GB RAM (Jade Z), 16GB 2GB RAM (Jade Z+)'}],
   'Main Camera': [{'Single': '13 MP, f/1.8, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '5 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP, EDR'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, gyro, proximity, compass'}],
   'Battery': [{'Type': 'Li-Po 2300 mAh, non-removable'},
    {'Stand-by': 'Up to 410 h (2G) / Up to 400 h (3G)'},
    {'Talk time': 'Up to 7 h (2G) / Up to 4 h (3G)'}],
   'Misc': [{'Colors': 'Black, White'}, {'Price': 'About 200 EUR'}]},
  'Liquid Z520': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'Speed': 'HSPA'}],
   'Launch': [{'Announced': '2015, March. Released 2015, April'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '154 x 86 x 6.4 mm (6.06 x 3.39 x 0.25 in)'},
    {'Weight': '118 g (4.16 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'TFT'},
    {'Size': '5.0 inches, 75.7 cm2(~57.1% screen-to-body ratio)'},
    {'Resolution': '480 x 854 pixels, 16:9 ratio (~187 ppi density)'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Mediatek MT6582M (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A7'},
    {'GPU': 'Mali-400MP2'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM, 16GB 2GB RAM'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': '1080p@30fps, 720p@60fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Po 2000 mAh, removable'}],
   'Misc': [{'Colors': 'Black, White'},
    {'Models': 'Z520'},
    {'Price': 'About 130 EUR'}]},
  'Liquid Z220': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 1900'},
    {'3G bands': 'HSDPA 850 / 1700(AWS)'},
    {'Speed': 'HSPA'}],
   'Launch': [{'Announced': '2015, March. Released 2015, April'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '125.3 x 64 x 9.6 mm (4.93 x 2.52 x 0.38 in)'},
    {'Weight': '120 g (4.23 oz)'},
    {'SIM': 'Single SIM (Mini-SIM) or Dual SIM (Mini-SIM/Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.0 inches, 45.5 cm2(~56.8% screen-to-body ratio)'},
    {'Resolution': '480 x 800 pixels, 5:3 ratio (~233 ppi density)'}],
   'Platform': [{'OS': 'Android 5.0 (Lollipop)'},
    {'Chipset': 'Qualcomm Snapdragon 200 (28 nm)'},
    {'CPU': 'Dual-core 1.2 GHz Cortex-A7'},
    {'GPU': 'Adreno 302'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM'},
    {'Internal': 'eMMC 4.5'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': '720p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Ion 1300 mAh, removable'}],
   'Misc': [{'Colors': 'Black, White'},
    {'Models': 'Z220'},
    {'Price': 'About 90 EUR'}]},
  'Liquid M220': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 1900'},
    {'3G bands': 'HSDPA 850 / 1700(AWS)'},
    {'Speed': 'HSPA 21.1/5.76 Mbps'}],
   'Launch': [{'Announced': '2015, March. Released 2015, May'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '124.9 x 64 x 9.6 mm (4.92 x 2.52 x 0.38 in)'},
    {'Weight': '119 g (4.20 oz)'},
    {'SIM': 'Single SIM (Mini-SIM) or Dual SIM (Mini-SIM/Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'TFT'},
    {'Size': '4.0 inches, 45.5 cm2(~57.0% screen-to-body ratio)'},
    {'Resolution': '480 x 800 pixels, 5:3 ratio (~233 ppi density)'}],
   'Platform': [{'OS': 'Microsoft Windows Phone 8.1'},
    {'Chipset': 'Qualcomm MSM8210 Snapdragon 200 (28 nm)'},
    {'CPU': 'Dual-core 1.2 GHz Cortex-A7'},
    {'GPU': 'Adreno 302'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '4GB 512MB RAM (M220), 8GB 1GB RAM (M220+)'},
    {'Internal': 'eMMC 4.5'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash, panorama'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 1300 mAh, removable'},
    {'Stand-by': 'Up to 200 h'},
    {'Talk time': 'Up to 4 h'}],
   'Misc': [{'Colors': 'Mystic Black, Pure White'},
    {'Price': 'About 80 EUR'}]},
  'Liquid Z410': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'4G bands': '1, 3, 7, 20'},
    {'4G bands': '2, 4, 7, 17'},
    {'Speed': 'HSPA, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2015, January. Released 2015, February'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '136 x 66.7 x 9.9 mm (5.35 x 2.63 x 0.39 in)'},
    {'Weight': '145 g (5.11 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.5 inches, 55.8 cm2(~61.5% screen-to-body ratio)'},
    {'Resolution': '540 x 960 pixels, 16:9 ratio (~245 ppi density)'}],
   'Platform': [{'OS': 'Android 4.4.4 (KitKat)'},
    {'Chipset': 'Mediatek MT6732M (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A53'},
    {'GPU': 'Mali-T760MP2'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM, 16GB 2GB RAM'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': 'Yes, dual shot'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, dual speakers'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Po 2000 mAh, removable'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 130 EUR'}]},
  'Liquid Jade S': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100 - EU'},
    {'4G bands': '3, 8, 28 - Taiwan'},
    {'4G bands': 'LTE 800 / 1800 / 2100 / 2600 - EU'},
    {'Speed': 'HSPA 21.1/5.76 Mbps, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2014, December. Released 2014, December'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '143 x 69 x 7.8 mm (5.63 x 2.72 x 0.31 in)'},
    {'Weight': '116 g (4.09 oz)'},
    {'SIM': 'Hybrid Dual SIM (Nano-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~69.8% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 3'}],
   'Platform': [{'OS': 'Android 4.4.4 (KitKat), upgradable to 5.1 (Lollipop)'},
    {'Chipset': 'Mediatek MT6752M'},
    {'CPU': 'Octa-core 1.5 GHz Cortex-A53'},
    {'GPU': 'Mali-T760MP2'}],
   'Memory': [{'Card slot': 'microSDHC (uses shared SIM slot)'},
    {'Internal': '16GB 2GB RAM'}],
   'Main Camera': [{'Single': '13 MP, f/1.8, AF'},
    {'Features': 'LED flash, HDR, panorama'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '5 MP, f/2.2'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP, EDR'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Ion 2300 mAh, non-removable'},
    {'Stand-by': 'Up to 250 h (3G)'},
    {'Talk time': 'Up to 6 h (3G)'}],
   'Misc': [{'Colors': 'Black, White, Red, Pink'},
    {'Models': 'S56'},
    {'Price': 'About 200 EUR'}],
   'Tests': [{'Performance': 'Basemark OS II 2.0: 725Basemark X: 3934'},
    {'Camera': 'Photo/Video'},
    {'Loudspeaker': 'Voice 60dB / Noise 56dB / Ring 60dB'},
    {'Battery (old)': 'Endurance rating 43h'}]},
  'Liquid Z500': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'Speed': 'HSPA'}],
   'Launch': [{'Announced': '2014, September. Released 2014, Septeber'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '145 x 73 x 8.8 mm (5.71 x 2.87 x 0.35 in)'},
    {'Weight': '150 g (5.29 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~65.1% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 3'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Mediatek MT6582 (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A7'},
    {'GPU': 'Mali-400MP2'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '4GB 1GB RAM, 16GB 2GB RAM'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP, EDR'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2000 mAh, non-removable'}],
   'Misc': [{'Colors': 'Aquamarine Green, Sandy Silver, Titanium Black'},
    {'SAR EU': '0.62 W/kg (head)'},
    {'Price': 'About 150 EUR'}]},
  'Liquid X1': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'4G bands': '3, 7'},
    {'Speed': 'HSPA, LTE'}],
   'Launch': [{'Announced': '2014, June. Released 2014, September'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '153.3 x 80.4 x 8.5 mm (6.04 x 3.17 x 0.33 in)'},
    {'Weight': '164 g (5.78 oz)'},
    {'SIM': 'Micro-SIM'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.7 inches, 89.6 cm2(~72.7% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~258 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 3'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Mediatek MT6592 (28 nm)'},
    {'CPU': 'Octa-core 1.7 GHz Cortex-A7'},
    {'GPU': 'Mali-450MP4'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '16GB 2GB RAM'}],
   'Main Camera': [{'Single': '13 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Ion 2700 mAh, removable'},
    {'Stand-by': 'Up to 250 h (3G)'},
    {'Talk time': 'Up to 6 h (3G)'}],
   'Misc': [{'Colors': 'Graphite black, Wine red'},
    {'Price': 'About 260 EUR'}]},
  'Liquid Jade': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'Speed': 'HSPA'}],
   'Launch': [{'Announced': '2014, June. Released 2014, July'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '140.5 x 69 x 7.5 mm (5.53 x 2.72 x 0.30 in)'},
    {'Weight': '110 g (3.88 oz)'},
    {'SIM': 'Hybrid Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~71.1% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'},
    {'Protection': 'Corning Gorilla Glass 3'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Mediatek MT6582 (28 nm)'},
    {'CPU': 'Quad-core 1.3 GHz Cortex-A7'},
    {'GPU': 'Mali-400MP2'}],
   'Memory': [{'Card slot': 'microSDHC (uses shared SIM slot)'},
    {'Internal': '8GB 1GB RAM, 16GB 2GB RAM'}],
   'Main Camera': [{'Single': '13 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity, compass'}],
   'Battery': [{'Type': 'Li-Ion 2100 mAh, non-removable'},
    {'Stand-by': 'Up to 330 h (3G)'},
    {'Talk time': 'Up to 5 h (3G)'}],
   'Misc': [{'Colors': 'Black, White, Green, Pink, Gray'},
    {'Price': 'About 180 EUR'}]},
  'Liquid E700': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 & SIM 3'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'Speed': 'HSPA'}],
   'Launch': [{'Announced': '2014, June. Released 2014, August'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '147.3 x 73 x 9.9 mm (5.80 x 2.87 x 0.39 in)'},
    {'Weight': '155 g (5.47 oz)'},
    {'SIM': 'Triple SIM (Micro-SIM)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~64.1% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~294 ppi density)'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Mediatek MT6582 (28 nm)'},
    {'CPU': 'Quad-core 1.2 GHz Cortex-A7'},
    {'GPU': 'Mali-400MP2'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '4GB 1GB RAM, 16GB 2GB RAM'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': '1080p@30fps'}],
   'Selfie camera': [{'Single': '2 MP'},
    {'Features': 'LED flash'},
    {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 3500 mAh, non-removable'},
    {'Stand-by': 'Up to 1440 h'},
    {'Talk time': 'Up to 24 h'},
    {'Music play': 'Up to 100 h'}],
   'Misc': [{'Colors': 'Titan Black, Burgundy Red'},
    {'Price': 'About 200 EUR'}]},
  'Liquid E600': {'Network': [{'Technology': 'GSM / HSPA / LTE'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'4G bands': '1, 3, 7, 20'},
    {'4G bands': 'LTE: 800 / 1700(AWS) 1800 / 1900 / 2100 / 2600 MHz'},
    {'Speed': 'HSPA 21.1/5.76 Mbps, LTE Cat4 150/50 Mbps'}],
   'Launch': [{'Announced': '2014, June. Released 2014, September'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '147 x 72 x 9.5 mm (5.79 x 2.83 x 0.37 in)'},
    {'Weight': '155 g (5.47 oz)'},
    {'SIM': 'Micro-SIM'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '5.0 inches, 68.9 cm2(~65.1% screen-to-body ratio)'},
    {'Resolution': '480 x 854 pixels, 16:9 ratio (~196 ppi density)'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Qualcomm MSM8926 Snapdragon 400 (28 nm)'},
    {'CPU': 'Quad-core 1.2 GHz Cortex-A7'},
    {'GPU': 'Adreno 305'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '4GB 1GB RAM, 16GB 2GB RAM'},
    {'Internal': 'eMMC 4.5'}],
   'Main Camera': [{'Single': '8 MP, AF'},
    {'Features': 'LED flash, HDR'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP or VGA'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2500 mAh, non-removable'},
    {'Talk time': 'Up to 6 h (3G)'}],
   'Misc': [{'Colors': 'Black, Green, Dark red'}, {'Price': 'About 200 EUR'}]},
  'Liquid Z200': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA 900 / 2100'},
    {'3G bands': 'HSDPA 850 / 900 / 1900 / 2100'},
    {'Speed': 'HSPA'}],
   'Launch': [{'Announced': '2014, June. Released 2014, September'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '126 x 65 x 10.3 mm (4.96 x 2.56 x 0.41 in)'},
    {'Weight': '130 g (4.59 oz)'},
    {'SIM': 'Single SIM (Mini-SIM) or Dual SIM (Mini-SIM/Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'TFT'},
    {'Size': '4.0 inches, 45.5 cm2(~55.6% screen-to-body ratio)'},
    {'Resolution': '480 x 800 pixels, 5:3 ratio (~233 ppi density)'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Mediatek MT6572M (28 nm)'},
    {'CPU': 'Dual-core 1.0 GHz Cortex-A7'},
    {'GPU': 'Mali-400'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '4GB 512MB RAM'}],
   'Main Camera': [{'Single': '2 MP'},
    {'Features': 'Panorama'},
    {'Video': 'Yes'}],
   'Selfie camera': [],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Ion 1300 mAh, removable'}],
   'Misc': [{'Colors': 'Titanium Black, Essential White, Fragrant Pink, Sky Blue, Sunshine Yellow'},
    {'Price': 'About 90 EUR'}]},
  'Iconia Tab 8 A1-840FHD': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2014, June. Released 2014, September'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '215 x 130 x 8.5 mm (8.46 x 5.12 x 0.33 in)'},
    {'Weight': '360 g (12.70 oz)'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '8.0 inches, 185.6 cm2(~66.4% screen-to-body ratio)'},
    {'Resolution': '1200 x 1920 pixels, 16:10 ratio (~283 ppi density)'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'},
    {'Chipset': 'Intel Atom Z3745'},
    {'CPU': 'Quad-core 1.86 GHz'},
    {'GPU': 'Intel Gen 7 (Ivy Bridge)'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '16GB 2GB RAM, 32GB 2GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 a/b/g/n, dual-band'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS, GLONASS'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}, {'Sensors': 'HDMI port'}],
   'Battery': [{'Type': 'Li-Ion 4600 mAh, non-removable'},
    {'Talk time': 'Up to 7 h 30 min (multimedia)'}],
   'Misc': [{'Colors': 'Silver'}, {'Price': 'About 200 EUR'}]},
  'Iconia Tab 7 A1-713': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 N/A'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2014, April. Released 2014, August'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '8.9 mm thickness'},
    {'Weight': '298 g (10.51 oz)'},
    {'SIM': 'Mini-SIM'}],
   'Display': [{'Type': 'TFT'},
    {'Size': '7.0 inches, 137.9 cm2'},
    {'Resolution': '600 x 1024 pixels, 16:9 ratio (~170 ppi density)'}],
   'Platform': [{'OS': 'Android 4.2.2 (Jelly Bean), upgradable to 4.4.2 (KitKat)'},
    {'CPU': 'Quad-core'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '16GB 1GB RAM'}],
   'Main Camera': [{'Single': '2 MP'}, {'Video': 'Yes'}],
   'Selfie camera': [{'Single': 'Yes'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Ion 3400 mAh, non-removable'},
    {'Talk time': 'Up to 6 h (multimedia)'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 110 EUR'}]},
  'Iconia Tab 7 A1-713HD': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 N/A'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2014, April. Released 2014, August'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '8.9 mm thickness'},
    {'Weight': '298 g (10.51 oz)'},
    {'SIM': 'Mini-SIM'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '7.0 inches, 142.1 cm2'},
    {'Resolution': '800 x 1280 pixels, 16:10 ratio (~216 ppi density)'}],
   'Platform': [{'OS': 'Android 4.4.2 (KitKat)'}, {'CPU': 'Quad-core'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '16GB 1GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': 'Yes'}],
   'Selfie camera': [{'Single': 'Yes'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Ion 3400 mAh, non-removable'},
    {'Talk time': 'Up to 6 h (multimedia)'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 150 EUR'}]},
  'Iconia One 7 B1-730': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2014, April. Released 2014, June'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '8.9 mm thickness'},
    {'Weight': '-'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '7.0 inches, 142.1 cm2'},
    {'Resolution': '800 x 1280 pixels, 16:10 ratio (~216 ppi density)'}],
   'Platform': [{'OS': 'Android 4.2.2 (Jelly Bean), upgradable to 4.4.2 (KitKat)'},
    {'Chipset': 'Intel Atom Z2560'},
    {'CPU': 'Dual-core 1.6 GHz'},
    {'GPU': 'PowerVR SGX544MP2'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM'}],
   'Main Camera': [{'Single': '5 MP'}, {'Video': 'Yes'}],
   'Selfie camera': [{'Single': 'Yes'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
    {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'No'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Ion 3700 mAh, non-removable'},
    {'Talk time': 'Up to 7 h (multimedia)'}],
   'Misc': [{'Colors': 'Red, white, black, blue, pink'},
    {'Price': 'About 90 EUR'}]},
  'Liquid E3 Duo Plus': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2014, Q3. Released 2014, Q3'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '138 x 69 x 8.9 mm (5.43 x 2.72 x 0.35 in)'},
    {'Weight': '-'},
    {'SIM': 'Dual SIM'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.7 inches, 60.9 cm2(~64.0% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~312 ppi density)'}],
   'Platform': [{'OS': 'Android 4.2.2 (Jelly Bean)'},
    {'Chipset': 'Mediatek MT6589 (28 nm)'},
    {'CPU': 'Quad-core 1.2 GHz Cortex-A7'},
    {'GPU': 'PowerVR SGX544'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '16GB 2GB RAM'}],
   'Main Camera': [{'Single': '13 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP'},
    {'Features': 'LED flash'},
    {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS, GLONASS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2000 mAh, non-removable'},
    {'Stand-by': 'Up to 260 h'},
    {'Talk time': 'Up to 5 h'}],
   'Misc': [{'Colors': 'Black'}, {'Price': 'About 240 EUR'}]},
  'Liquid E3': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2014, February. Released 2014, April'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '136 x 68 x 9 mm (5.35 x 2.68 x 0.35 in)'},
    {'Weight': '134 g (4.73 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'IPS LCD'},
    {'Size': '4.7 inches, 60.9 cm2(~65.8% screen-to-body ratio)'},
    {'Resolution': '720 x 1280 pixels, 16:9 ratio (~312 ppi density)'}],
   'Platform': [{'OS': 'Android 4.2.2 (Jelly Bean)'},
    {'Chipset': 'Mediatek MT6589 (28 nm)'},
    {'CPU': 'Quad-core 1.2 GHz Cortex-A7'},
    {'GPU': 'PowerVR SGX544'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '4GB 1GB RAM'}],
   'Main Camera': [{'Single': '13 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': 'Yes'}],
   'Selfie camera': [{'Single': '2 MP'},
    {'Features': 'LED flash'},
    {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS, GLONASS'},
    {'NFC': 'No'},
    {'Radio': 'FM radio'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 2500 mAh, non-removable'},
    {'Stand-by': 'Up to 260 h'},
    {'Talk time': 'Up to 5 h'}],
   'Misc': [{'Colors': 'Black/White'}, {'Price': 'About 200 EUR'}]},
  'Liquid Z4': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (dual-SIM model only)'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2014, February. Released 2014, April'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '124 x 64 x 9.7 mm (4.88 x 2.52 x 0.38 in)'},
    {'Weight': '122 g (4.30 oz)'},
    {'SIM': 'Single SIM (Micro-SIM) or Dual SIM (Micro-SIM, dual stand-by)'}],
   'Display': [{'Type': 'TFT'},
    {'Size': '4.0 inches, 45.5 cm2(~57.4% screen-to-body ratio)'},
    {'Resolution': '480 x 800 pixels, 5:3 ratio (~233 ppi density)'}],
   'Platform': [{'OS': 'Android 4.2.2 (Jelly Bean)'},
    {'Chipset': 'Mediatek MT6572 (28 nm)'},
    {'CPU': 'Dual-core 1.3 GHz Cortex-A7'},
    {'GPU': 'Mali-400'}],
   'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
    {'Internal': '4GB 512MB RAM'}],
   'Main Camera': [{'Single': '5 MP, AF'},
    {'Features': 'LED flash'},
    {'Video': '720p@30fps'}],
   'Selfie camera': [{'Single': '1.3 MP'}, {'Video': ''}],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n, hotspot'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'No'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer, proximity'}],
   'Battery': [{'Type': 'Li-Ion 1630 mAh, removable'}],
   'Misc': [{'Colors': 'Black/White'}, {'Price': 'About 100 EUR'}]},
  'Iconia B1-721': {'Network': [{'Technology': 'GSM / HSPA'},
    {'2G bands': 'GSM 850 / 900 / 1800 / 1900'},
    {'3G bands': 'HSDPA'},
    {'Speed': 'HSPA'},
    {'GPRS': 'Yes'},
    {'EDGE': 'Yes'}],
   'Launch': [{'Announced': '2014, January. Released 2014, January'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '199 x 122.3 x 11.4 mm (7.83 x 4.81 x 0.45 in)'},
    {'Weight': '323 g (11.39 oz)'},
    {'SIM': 'Yes'}],
   'Display': [{'Type': 'TFT'},
    {'Size': '7.0 inches, 137.9 cm2(~56.7% screen-to-body ratio)'},
    {'Resolution': '600 x 1024 pixels, 16:9 ratio (~170 ppi density)'}],
   'Platform': [{'OS': 'Android 4.2 (Jelly Bean)'},
    {'Chipset': 'Mediatek MT8312 (28 nm)'},
    {'CPU': 'Dual-core 1.3 GHz Cortex-A7'},
    {'GPU': 'Mali-400'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '16GB 1GB RAM'}],
   'Main Camera': [{'Single': 'VGA'}, {'Video': 'Yes'}],
   'Selfie camera': [],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': 'Yes'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Po 2955 mAh, non-removable'}],
   'Misc': [{'Colors': 'Black/Red'}, {'Price': 'About 130 EUR'}]},
  'Iconia B1-720': {'Network': [{'Technology': 'No cellular connectivity'},
    {'2G bands': 'N/A'},
    {'GPRS': 'No'},
    {'EDGE': 'No'}],
   'Launch': [{'Announced': '2014, January. Released 2014, January'},
    {'Status': 'Discontinued'}],
   'Body': [{'Dimensions': '198.1 x 121.9 x 10.2 mm (7.80 x 4.80 x 0.40 in)'},
    {'Weight': '315 g (11.11 oz)'},
    {'SIM': 'No'}],
   'Display': [{'Type': 'TFT'},
    {'Size': '7.0 inches, 137.9 cm2(~57.1% screen-to-body ratio)'},
    {'Resolution': '600 x 1024 pixels, 16:9 ratio (~170 ppi density)'}],
   'Platform': [{'OS': 'Android 4.2 (Jelly Bean)'},
    {'Chipset': 'Mediatek MT8111'},
    {'CPU': 'Dual-core 1.3 GHz'},
    {'GPU': 'PowerVR SGX531'}],
   'Memory': [{'Card slot': 'microSDHC (dedicated slot)'},
    {'Internal': '8GB 1GB RAM, 16GB 1GB RAM'}],
   'Main Camera': [{'Single': 'VGA'}, {'Video': 'Yes'}],
   'Selfie camera': [],
   'Sound': [{'Loudspeaker': 'Yes'}, {'3.5mm jack': 'Yes'}],
   'Comms': [{'WLAN': 'Wi-Fi 802.11 b/g/n'},
    {'Bluetooth': '4.0, A2DP'},
    {'Positioning': 'GPS'},
    {'NFC': 'No'},
    {'Radio': 'No'},
    {'USB': 'microUSB 2.0'}],
   'Features': [{'Sensors': 'Accelerometer'}],
   'Battery': [{'Type': 'Li-Po 2955 mAh, non-removable'},
    {'Talk time': 'Up to 5 h (multimedia)'}],
   'Misc': [{'Colors': 'Black, Gold'}, {'Price': 'About 100 EUR'}]},
  'Iconia A1-830': {},
  'Liquid Z5': {}}}


{'Chromebook Tab 10': {'Network': [{'Technology': 'No cellular connectivity'},
   {'2G bands': 'N/A'},
   {'GPRS': 'No'},
   {'EDGE': 'No'}],
  'Launch': [{'Announced': '2018, March'},
   {'Status': 'Available. Released 2018, July'}],
  'Body': [{'Dimensions': '238.3 x 172.2 x 9.9 mm (9.38 x 6.78 x 0.39 in)'},
   {'Weight': '544.3 g (1.20 lb)'},
   {'SIM': 'No'},
   {'SIM': 'Stylus'}],
  'Display': [{'Type': 'IPS LCD'},
   {'Size': '9.7 inches, 291.4 cm2(~71.0% screen-to-body ratio)'},
   {'Resolution': '1536 x 2048 pixels, 4:3 ratio (~264 ppi density)'}],
  'Platform': [{'OS': 'Chrome OS'},
   {'Chipset': 'Rockchip RK3399'},
   {'CPU': 'Hexa-core (4x Cortex-A53 & 2x Cortex-A72)'}],
  'Memory': [{'Card slot': 'microSDXC (dedicated slot)'},
   {'Internal': '32GB 4GB RAM'}],
  'Main Camera': [{'Single': '5 MP'}, {'Video': '720p'}],
  'Selfie camera': [{'Single': '2 MP'}, {'Video': ''}],
  'Sound': [{'Loudspeaker': 'Yes, with stereo speakers'},
   {'3.5mm jack': 'Yes'}],
  'Comms': [{'WLA